In [ ]:
from IPython.display import display
import _functions
from rs_classes import async_request_client as async_client

# #Initialize client
client = async_client.AsyncRequestClient('', '')

# Initialize the set_widgets list
url_input, bool_toggle, dropdown = _functions.create_input_widgets()
display(dropdown, url_input, bool_toggle)

### Put Token, Query and list of Fields to the next cell and run it. 

In [ ]:
token_input = ""
query = {
    "query": {
        "$and": [
            # {
            #     "queue": {
            #         "$in": [
            #             "https://elis.rossum.ai/api/v1/queues/XXXXXX",
            #             "https://elis.rossum.ai/api/v1/queues/XXXXXX",
            #             "https://elis.rossum.ai/api/v1/queues/XXXXXX"
            #         ]
            #     }
            # },
            # {
            #     "field.sender_name.string": {
            #         "$eq": "Contoso Ltd"
            #     }
            # },
            {
                "status": {
                    "$in": [
                        "confirmed",
                        "exported",
                        "to_review"
                    ]
                }
            }
        ]
    }
}
field_id = ['date_issue', 'document_id']
await _functions.process_annotations(client, token_input, url_input, query, field_id, bool_toggle, dropdown)